In [3]:
#my client credentials for Spotify API (NELLE'S!!!)
Client_ID = "4555887f29754e4188258d4719c0221c"
Client_Secret = "4e400da7e73a4e9eaa442da8c06922af"

#10,000 songs, seemingly randomly chosen
big_playlist_url = "https://open.spotify.com/playlist/5S8SJdl1BDc0ugpkEvFsIL?si=95d1072596dd4078"
big_playlist_id = "5S8SJdl1BDc0ugpkEvFsIL?si=95d1072596dd4078"

In [36]:
#import needed packages
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time

#intialize the Spotify webscraper and input into "spotipy"
client_credentials_manager = SpotifyClientCredentials(client_id=Client_ID, client_secret=Client_Secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [35]:
#intialize columns of our future pandas dataframe
titles = []
preview_urls = []
popularity_scores =[]
artist_urls = []
artist_pops = []
artist_names = []
song_urls = []
track_ids = []

#Because the maximum batch size for this API is 100, that is how we will iterate through the playlist
for i in range(0,10000,100):
    #request the i-th set of 100 songs from the playlist
    curr_playlist = sp.playlist_items(big_playlist_url, offset = i, additional_types = ['track'])
    time.sleep(1)
    #loop though all 100 songs returned
    for song_dict in curr_playlist['items']:
        #append all future df attributes we are going to need
        titles.append(song_dict['track']['name'])
        preview_urls.append(song_dict['track']['preview_url'])
        popularity_scores.append(song_dict['track']['popularity'])
        #pull the artist id and append to batch list
        try:
            artist_id = song_dict['track']['artists'][0]['external_urls']['spotify']
        except:
            artist_id = "https://open.spotify.com/artist/240wlM8vDrf6S4zCyzGj2W?si=SxFlZdThTCGoD5n7DrewTQ"
        artist_urls.append(artist_id)
        #append track id for future use in audio feature extraction
        track_ids.append(song_dict['track']['id'])
        
print("here!")
#now iterate through the list of artist id's
for i in range(0,10000,50):
    #take a batch of size 50 to request
    itter_list = artist_urls[i:i+50]
    artist_list = sp.artists(itter_list)
    time.sleep(1)
    #append these values to our lists
    for dict in artist_list['artists']:
        artist_pops.append(dict['popularity'])
        artist_names.append(dict['name'])


#create our dataframe, control song popularity according to artist popularity
big_df = pd.DataFrame({'title':titles,'track_id':track_ids,'preview_url':preview_urls,'song_popularity':popularity_scores,'artist_popularity':artist_pops,'artist':artist_names})
big_df.to_csv('songs_artists.csv')

here!


In [5]:
#Data Cleaning for CNN model using spectrograms
big_df = pd.read_csv('songs_artists.csv')
clean_df = big_df[big_df.song_popularity != 0][big_df.artist_popularity != 0][big_df.preview_url.notna()]
clean_df['normalized_pop'] = clean_df['song_popularity']/clean_df['artist_popularity']
final_df = clean_df.filter(items = ['title','preview_url','song_popularity','artist_popularity','artist','normalized_pop'])
final_df.to_csv('final.csv', index = 3)
final_df = final_df[['title','preview_url','song_popularity','artist_popularity','artist','normalized_pop']]
boom_mean = final_df.normalized_pop.mean()
final_df['boom_bust'] = final_df['normalized_pop'] > boom_mean
final_df.to_csv('final.csv', index = 3)

C:\Users\william.niven\AppData\Local\Temp\ipykernel_26156\759958213.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  clean_df = big_df[big_df.song_popularity != 0][big_df.artist_popularity != 0][big_df.preview_url.notna()]
C:\Users\william.niven\AppData\Local\Temp\ipykernel_26156\759958213.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  clean_df = big_df[big_df.song_popularity != 0][big_df.artist_popularity != 0][big_df.preview_url.notna()]
